In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)


In [15]:
data = pd.read_csv("../data/extruder_recipe_unaggregated.csv",decimal=".",sep=";")
data.columns

/var/folders/wb/ds0bcygd1bz7hr_3nm8mcg_c0000gn/T/ipykernel_12673/1124218396.py:1: DtypeWarning: Columns (5,6,7,8,9,23,24,25,26,27,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/extruder_recipe_unaggregated.csv",decimal=".",sep=";")


Index(['local_time', 'product_code [None]', 'production_order_code [None]',
       'Extruder A HK Material [None]', 'Extruder A HK Anteil [%]',
       'Extruder A NK 1 Material [None]', 'Extruder A NK 2 Material [None]',
       'Extruder A NK 3 Material [None]', 'Extruder A NK 4 Material [None]',
       'Extruder A NK 5 Material [None]', 'Extruder A NK 1 Anteil (Soll) [%]',
       'Extruder A NK 1 Anteil [%]', 'Extruder A NK 2 Anteil (Soll) [%]',
       'Extruder A NK 2 Anteil [%]', 'Extruder A NK 3 Anteil (Soll) [%]',
       'Extruder A NK 3 Anteil [%]', 'Extruder A NK 4 Anteil (Soll) [%]',
       'Extruder A NK 4 Anteil [%]', 'Extruder A NK 5 Anteil (Soll) [%]',
       'Extruder A NK 5 Anteil [%]', 'Extruder A Massetemperatur [°C]',
       'Extruder B HK Material [None]', 'Extruder B HK Anteil [%]',
       'Extruder B NK 1 Material [None]', 'Extruder B NK 2 Material [None]',
       'Extruder B NK 3 Material [None]', 'Extruder B NK 4 Material [None]',
       'Extruder B NK 5 Material 

In [3]:
'''Drop NaNs in product_code'''
data.dropna(subset=["product_code [None]"],inplace=True)

'''Fill NaNs with 0'''
data = data.fillna(0)
data.isna().sum()

'''Convert Floats to int for better performance'''
data_converted = data.astype({col: 'int64' for col in data.select_dtypes(exclude='object').columns})
data_converted.dtypes
data_converted.head(10)

'''Export csv'''
#data_converted.to_csv("extruder_recipe_converted.csv")


'''Remove all non-recipe rows except product_code'''
data_removed = data_converted.drop(["local_time","production_order_code [None]","Extruder C Massetemperatur [°C]","Extruder A Massetemperatur [°C]","Extruder B Massetemperatur [°C]"],axis=1)
''' Remove all except 6 rows'''
input_rows = ['Extruder A HK Material [None]', 'Extruder A HK Anteil [%]','Extruder B HK Material [None]', 'Extruder B HK Anteil [%]','Extruder C HK Material [None]', 'Extruder C HK Anteil [%]','product_code [None]']
data_removed = data_converted[input_rows]
print(data_removed.columns)


'''strip all strings'''
data_removed = data_removed.applymap(lambda x: x.strip().replace(" ", "") if isinstance(x, str) else x)

#data_removed

Index(['Extruder A HK Material [None]', 'Extruder A HK Anteil [%]',
       'Extruder B HK Material [None]', 'Extruder B HK Anteil [%]',
       'Extruder C HK Material [None]', 'Extruder C HK Anteil [%]',
       'product_code [None]'],
      dtype='object')


In [14]:
'''group_by all columns aka create trainingsset'''



grouped_df = data_removed.groupby(list(data_removed.columns)).size().reset_index(name="Count")
#grouped_df.sort_values("product_code [None]",ascending=False)
#grouped_df.loc[grouped_df["product_code [None]"] == 94705967]

grouped_df.shape

'''check how many different materials are used'''
mat_list = set()
for col in [x for x in grouped_df.columns if "Material" in x]:
    mat_list.update(set(grouped_df[col]))

In [8]:
'''Do the machine learning part'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from category_encoders import HashingEncoder


'''Create Train set'''
X = data.drop("Count",axis=1)

# Take out cluster test values
#cluster1 = [94107836, 94163716, 94165248, 94165975, 94169548]
cluster3 = [94102432,94105895,94106158]
cluster3_1 = [94161726]
#cluster2 = [94169338,94169708,94170124]
#outlier_cluster = [42121147, 58211230, 58211728, 94106740, 94163071, 94164272, 94164559, 94165695, 94166021, 94167918,94169960, 94507691, 94704621, 94705909]
X = X[~X["product_code [None]"].isin([94161726])]
y = X["product_code [None]"]
X = X.drop("product_code [None]",axis=1)

categorical_cols_train = [col for col in X.columns if "Material" in col]
numeric_cols_train = [col for col in X.columns if X[col].dtype != 'object' and not 'product' in col]

X[numeric_cols_train] = X[numeric_cols_train].astype("int64")

X[categorical_cols_train] = X[categorical_cols_train].astype("str")


'''Fit OneHot Encoding to Train testset'''
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X[categorical_cols_train])
Train_Prep = ohe.transform(X[categorical_cols_train])
Train_Prep = pd.DataFrame(Train_Prep.toarray())
print(Train_Prep.shape)

'''Merge categorical with integer rows'''
X_num_train = X[numeric_cols_train]
# Index Reseting is important
X_num_train.reset_index(drop=True, inplace=True)
Train_Prep.reset_index(drop=True, inplace=True)

X_encoded = pd.concat([X_num_train,Train_Prep],axis=1,ignore_index=True)
print(X_encoded.shape)
print("Preprocessing Done...")


'''Train'''
X_train, X_valid, y_train, y_valid = train_test_split(X_encoded, y, test_size=0.2, random_state=22)

rf_classifier = RandomForestClassifier(verbose=1,n_estimators = 100,n_jobs=-1)
#catboost_classifier = CatBoostClassifier(task_type='GPU',iterations=1000)

print("Start Fit...")
rf_classifier.fit(X_train, y_train)
#catboost_classifier.fit(X_train, y_train)

print("Start Prediction...")
y_pred = rf_classifier.predict(X_valid)
#y_pred = catboost_classifier.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)
report = classification_report(y_valid, y_pred)
print("Classification Report:")
print(report)

print("Validation Accuracy:", accuracy)


(91762, 254)
(91762, 288)
Preprocessing Done...
Start Fit...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.6s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.


Start Prediction...


[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.3s


Classification Report:
              precision    recall  f1-score   support

    42120873       0.98      1.00      0.99        86
    42121147       0.98      1.00      0.99        50
    42121150       1.00      1.00      1.00        34
    58210594       1.00      1.00      1.00         3
    58210716       1.00      0.96      0.98        27
    58210717       1.00      1.00      1.00        14
    58210718       1.00      1.00      1.00         8
    58210900       1.00      1.00      1.00         4
    58211230       1.00      1.00      1.00        53
    58211728       0.93      1.00      0.96        13
    94102432       1.00      1.00      1.00       208
    94105895       1.00      0.99      0.99        79
    94106158       1.00      1.00      1.00       133
    94106740       1.00      1.00      1.00        21
    94107836       1.00      1.00      1.00        67
    94161275       1.00      1.00      1.00        87
    94161282       1.00      1.00      1.00       161
    

[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.8s finished


In [9]:
'''Predict with Tree'''

'''Create Prediciton Set'''
values_to_predict = data.drop("Count",axis=1)


values_to_predict = values_to_predict[values_to_predict["product_code [None]"].isin(cluster3)]
values_to_predict = values_to_predict.drop("product_code [None]",axis=1)

categorical_cols_pred = [col for col in values_to_predict.columns if "Material" in col]
numeric_cols_pred = [col for col in values_to_predict.columns if values_to_predict[col].dtype != 'object']



values_to_predict[numeric_cols_pred] = values_to_predict[numeric_cols_pred].astype("int64")

values_to_predict[categorical_cols_pred] = values_to_predict[categorical_cols_pred].astype("str")

input_dict = {
    'Extruder A HK Material [None]': 'FT5230',
    'Extruder A HK Anteil [%]': 73,
    'Extruder B HK Material [None]': 'BA110CF',
    'Extruder B HK Anteil [%]': 45,
    'Extruder C HK Material [None]': 'FT5230',
    'Extruder C HK Anteil [%]': 79
}

#df = pd.DataFrame(input_dict,index=[0])

'''Fit OneHot Encoding to Prediction set'''
Pred_Prep = ohe.transform(values_to_predict[categorical_cols_pred])
Pred_Prep = pd.DataFrame(Pred_Prep.toarray())
print(values_to_predict[numeric_cols_pred].shape)

'''Merge categorical with integer rows'''
Pred_num = values_to_predict[numeric_cols_pred]
# Index Reseting is important
Pred_num.reset_index(drop=True, inplace=True)
Pred_Prep.reset_index(drop=True, inplace=True)

Pred_encoded = pd.concat([Pred_num,Pred_Prep],axis=1,ignore_index=True)
print(Pred_encoded)

''''''

'''Predict'''
output = rf_classifier.predict(Pred_encoded)
print(output)
print(sum([1 if x in [94704621, 94705909] else 0 for x in output]))
values_to_predict

(2055, 34)
       0    1    2    3    4    5    6    7    8    9    10   11   12   13   \
0     1563    0    0    0    0    0    0    0    0    0    1    0    0    0   
1     1564    0    0    0    0    0    0    0    0    0    1    0   99    0   
2     1565    0    0    0    0    0    8    0    0    0    1    0    0    0   
3     1566    0    0    0    2    0    8    0    0    0    1    0    0    0   
4     1567    0    0    0    2    0    8    0    0    0    7    0    0    0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2050  3613   91   10    8    0    0    0    0    0    0    1    1   96   10   
2051  3614   91   10    8    0    0    0    0    0    0   10    1   96   10   
2052  3615   92   10    6    0    0    0    0    0    0    1    1   96   10   
2053  3616   99   40    0    0    0    0    0    0    0    1    1   83   10   
2054  3617  100   10    0    0    0    0    0    0    0    1    0  100   10   

      14   15   16   17   18   19   20  

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.1s finished


,Unnamed: 0,Extruder A HK Material [None],Extruder A HK Anteil [%],Extruder A NK 1 Material [None],Extruder A NK 2 Material [None],Extruder A NK 3 Material [None],Extruder A NK 4 Material [None],Extruder A NK 5 Material [None],Extruder A NK 1 Anteil (Soll) [%],Extruder A NK 1 Anteil [%],Extruder A NK 2 Anteil (Soll) [%],Extruder A NK 2 Anteil [%],Extruder A NK 3 Anteil (Soll) [%],Extruder A NK 3 Anteil [%],Extruder A NK 4 Anteil (Soll) [%],Extruder A NK 4 Anteil [%],Extruder A NK 5 Anteil (Soll) [%],Extruder A NK 5 Anteil [%],Extruder B HK Material [None],Extruder B HK Anteil [%],Extruder B NK 1 Material [None],Extruder B NK 2 Material [None],Extruder B NK 3 Material [None],Extruder B NK 4 Material [None],Extruder B NK 5 Material [None],Extruder B NK 1 Anteil (Soll) [%],Extruder B NK 1 Anteil [%],Extruder B NK 2 Anteil (Soll) [%],Extruder B NK 2 Anteil [%],Extruder B NK 3 Anteil (Soll) [%],Extruder B NK 3 Anteil [%],Extruder B NK 4 Anteil (Soll) [%],Extruder B NK 4 Anteil [%],Extruder B NK 5 Anteil (Soll) [%],Extruder B NK 5 Anteil [%],Extruder C HK Material [None],Extruder C HK Anteil [%],Extruder C NK 1 Material [None],Extruder C NK 2 Material [None],Extruder C NK 3 Material [None],Extruder C NK 4 Material [None],Extruder C NK 5 Material [None],Extruder C NK 1 Anteil (Soll) [%],Extruder C NK 1 Anteil [%],Extruder C NK 2 Anteil (Soll) [%],Extruder C NK 2 Anteil [%],Extruder C NK 3 Anteil (Soll) [%],Extruder C NK 3 Anteil [%],Extruder C NK 4 Anteil (Soll) [%],Extruder C NK 4 Anteil [%],Extruder C NK 5 Anteil (Soll) [%],Extruder C NK 5 Anteil [%]
1563,1563,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,0,0,0,0,0,0,1,0,1018MA,0,0,10007,190900-S,0,RG,0,0,0,0,0,0,0,0,0,0,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,0,0,0,0,0,0,1,0
1564,1564,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,0,0,0,0,0,0,1,0,1018MA,99,0,10007,190900-S,0,RG,0,0,0,0,0,0,0,0,0,0,1018MA,99,0,10007.0,190900-S,0,H0039,0,0,0,0,0,0,0,0,1,1
1565,1565,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,0,0,8,0,0,0,1,0,1018MA,0,0,10007,190900-S,0,RG,0,0,0,0,8,0,0,0,0,0,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,0,0,8,0,0,0,1,0
1566,1566,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,2,0,8,0,0,0,1,0,1018MA,0,0,10007,190900-S,0,RG,0,0,2,0,8,0,0,0,0,0,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,2,0,8,0,0,0,1,0
1567,1567,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,2,0,8,0,0,0,7,0,1018MA,0,0,10007,190900-S,0,RG,0,0,2,0,8,0,0,0,0,0,1018MA,0,0,10007.0,190900-S,0,H0039,0,0,2,0,8,0,0,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,3613,118NJ,91,FT5230,0,0,nan,H0039,10,8,0,0,0,0,0,0,1,1,118NJ,96,FT5230,0,0,RG,0,10,3,0,0,0,0,0,0,0,0,HP1018BN,93,FT5230,0,102122,nan,H0039,10,4,0,0,3,1,0,0,1,1
3614,3614,118NJ,91,FT5230,0,0,nan,H0039,10,8,0,0,0,0,0,0,10,1,118NJ,96,FT5230,0,0,RG,0,10,3,0,0,0,0,0,0,0,0,HP1018BN,93,FT5230,0,102122,nan,H0039,10,4,0,0,3,1,0,0,10,1
3615,3615,118NJ,92,FT5230,0,0,nan,H0039,10,6,0,0,0,0,0,0,1,1,118NJ,96,FT5230,0,0,RG,0,10,3,0,0,0,0,0,0,0,0,HP1018BN,93,FT5230,0,102122,nan,H0039,10,4,0,0,3,1,0,0,1,1
3616,3616,118NJ,99,FT5230,0,0,nan,H0039,40,0,0,0,0,0,0,0,1,1,118NJ,83,FT5230,0,0,RG,0,10,16,0,0,0,0,0,0,0,0,HP1018BN,58,FT5230,0,102122,nan,H0039,40,38,0,0,3,2,0,0,1,1
